<a href="https://colab.research.google.com/github/billibilbi/almost_legal_links/blob/main/torchtraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
   

In [46]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer



dataset = load_dataset("csv", data_files="/content/ccass_rapprochements_sommaires_with_sources.csv")


In [47]:
# Create train and validation splits
dataset = dataset['train'].train_test_split(test_size=0.2)  # Split the 'train' split
dataset['validation'] = dataset.pop('test')  # Rename 'test' to 'validation'

In [48]:
tokenized_datasets = tokenized_datasets['train'].train_test_split(test_size=0.2)
tokenized_datasets['validation'] = tokenized_datasets.pop('test')

In [49]:
from transformers import AutoTokenizer

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [50]:
def tokenize_function(examples):
    return tokenizer(
        examples['SOMMAIRE_SOURCE'],
        examples['JUDILIBRE_SOURCE'],
        examples['SOMMAIRE_RAPPROCHEMENT'],
        examples['JUDILIBRE_RAPPROCHEMENT'],
        padding='max_length',
        truncation=True
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7840 [00:00<?, ? examples/s]

Map:   0%|          | 0/1961 [00:00<?, ? examples/s]

In [51]:
print(tokenized_datasets)
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['SOMMAIRE_SOURCE', 'JUDILIBRE_SOURCE', 'SOMMAIRE_RAPPROCHEMENT', 'JUDILIBRE_RAPPROCHEMENT', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7840
    })
    validation: Dataset({
        features: ['SOMMAIRE_SOURCE', 'JUDILIBRE_SOURCE', 'SOMMAIRE_RAPPROCHEMENT', 'JUDILIBRE_RAPPROCHEMENT', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1961
    })
})
DatasetDict({
    train: Dataset({
        features: ['SOMMAIRE_SOURCE', 'JUDILIBRE_SOURCE', 'SOMMAIRE_RAPPROCHEMENT', 'JUDILIBRE_RAPPROCHEMENT'],
        num_rows: 7840
    })
    validation: Dataset({
        features: ['SOMMAIRE_SOURCE', 'JUDILIBRE_SOURCE', 'SOMMAIRE_RAPPROCHEMENT', 'JUDILIBRE_RAPPROCHEMENT'],
        num_rows: 1961
    })
})


In [52]:
from transformers import AutoModelForSequenceClassification

# Number of unique labels in your dataset
num_labels = len(dataset['train'].unique('SOMMAIRE_RAPPROCHEMENT'))

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Flattening the indices:   0%|          | 0/7840 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AFTER THIS


In [60]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [54]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

The error message "ValueError: Expected input batch_size (8) to match target batch_size (4096)" indicates a mismatch between the batch size of your input data and the batch size of your target labels during training.

This is likely happening because the tokenize_function is tokenizing four separate text columns and concatenating them into a single input sequence. This creates an input with a batch size of 8 (as defined in TrainingArguments). However, the target labels ('SOMMAIRE_RAPPROCHEMENT') still have a batch size corresponding to the original dataset, which is 4096 in this case. The model expects the input and target batch sizes to match, leading to the ValueError.

In [58]:
def tokenize_function(examples):
    return tokenizer(
        examples['SOMMAIRE_SOURCE'],
        padding='max_length',
        truncation=True
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7840 [00:00<?, ? examples/s]

Map:   0%|          | 0/1961 [00:00<?, ? examples/s]

In [61]:
trainer.train()

AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.